In [ ]:
# Importing Libraries
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random
from datetime import datetime
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Reshape,
    UpSampling2D,
    Conv2D,
    BatchNormalization,
    Add,
    Layer
)
from tensorflow.keras.layers import (
    LeakyReLU,
    Dropout,
    Flatten,
    Activation,
)
import warnings


# Ignore warnings
warnings.filterwarnings("ignore")

In [70]:
# Set the random seed for reproducibility
SEED = 1618

# Set Python's built-in random seed
random.seed(SEED)

# Set NumPy random seed
np.random.seed(SEED)

# Set TensorFlow random seed
tf.random.set_seed(SEED)

# Make sure any new random state is also deterministic
os.environ['PYTHONHASHSEED'] = str(SEED)

In [71]:
# Directories
INPUT_DIR = "../../data_resized"
now_fmt = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_DIR = f"../../data_generated/{now_fmt}"
os.mkdir(OUTPUT_DIR)

# Hyperparameters
BATCH_SIZE = 64
IMG_HEIGHT = 256
IMG_WIDTH = 256
EPOCHS = 200
LATENT_DIM = 128
NUM_EXAMPLES_TO_GENERATE = 16

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(
    INPUT_DIR,
    labels=None,  # Set to None as you're not using labels
    label_mode=None,
    image_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=True,  # Enable shuffling for better training
)

# Apply rescaling
data = data.map(lambda x: x / 127.5 - 1)

In [ ]:
# Define a custom residual block as a layer
class ResidualBlock(Layer):
    def __init__(self, filters, kernel_size=3):
        super(ResidualBlock, self).__init__()
        self.conv1 = Conv2D(filters, kernel_size=kernel_size, padding="same")
        self.batchnorm1 = BatchNormalization()
        self.activation1 = Activation("relu")
        self.conv2 = Conv2D(filters, kernel_size=kernel_size, padding="same")
        self.batchnorm2 = BatchNormalization()
        self.activation2 = Activation("relu")

    def call(self, inputs, training=False):
        x = self.conv1(inputs)
        x = self.batchnorm1(x, training=training)
        x = self.activation1(x)
        x = self.conv2(x)
        x = self.batchnorm2(x, training=training)
        x = Add()([inputs, x])
        return self.activation2(x)

generator = Sequential()

# First layer: fully connected
generator.add(Dense(8 * 8 * 512, input_dim=LATENT_DIM))
generator.add(Reshape((8, 8, 512)))
generator.add(BatchNormalization())
generator.add(Activation("relu"))

# First UpSampling Block: Upsample to 16x16
generator.add(UpSampling2D(size=(2, 2)))  # Use UpSampling instead of Conv2DTranspose
generator.add(Conv2D(256, kernel_size=5, padding="same"))  # Convolution after upsampling
generator.add(BatchNormalization())
generator.add(Activation("relu"))


# Second UpSampling Block: Upsample to 32x32
generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(128, kernel_size=5, padding="same"))
generator.add(BatchNormalization())
generator.add(Activation("relu"))

# Third UpSampling Block: Upsample to 64x64
generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(64, kernel_size=5, padding="same"))
generator.add(BatchNormalization())
generator.add(Activation("relu"))

# Fourth UpSampling Block: Upsample to 128x128
generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(32, kernel_size=5, padding="same"))
generator.add(BatchNormalization())
generator.add(Activation("relu"))

# Final UpSampling Block: Upsample to 256x256
generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(16, kernel_size=5, padding="same"))
generator.add(BatchNormalization())
generator.add(Activation("relu"))

# Final layer: Convolution to generate the 256x256x3 image
generator.add(Conv2D(3, kernel_size=5, padding="same"))
generator.add(Activation("tanh"))  # Output values are in the range [-1, 1]

generator.summary()


In [ ]:
discriminator = Sequential()

# First Convolutional Block: Downsample to 128x128
discriminator.add(Conv2D(64, kernel_size=5, strides=2, padding="same", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.3))

# Second Convolutional Block: Downsample to 64x64
discriminator.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.3))

# Third Convolutional Block: Downsample to 32x32
discriminator.add(Conv2D(256, kernel_size=5, strides=2, padding="same"))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.3))

# Fourth Convolutional Block: Downsample to 16x16
discriminator.add(Conv2D(512, kernel_size=5, strides=2, padding="same"))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.3))

# # Fifth Convolutional Block (New): Downsample to 8x8
discriminator.add(Conv2D(512, kernel_size=5, strides=2, padding="same"))
discriminator.add(BatchNormalization())
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dropout(0.3))

# Flattening and Output
discriminator.add(Flatten())
discriminator.add(Dense(1, activation="sigmoid"))

# Display model summary
discriminator.summary()

In [75]:
class GAN(tf.keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(GAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = tf.keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = tf.keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    @tf.function
    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        seed = tf.random.normal(shape=(batch_size, self.latent_dim))
        # Decode them to fake images
        generated_images = self.generator(seed)
        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Assemble labels discriminating real from fake images
        labels = tf.concat(
            [tf.zeros((batch_size, 1)), tf.ones((batch_size, 1)) * 0.9], axis=0
        )

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        seed = tf.random.normal(shape=(batch_size, self.latent_dim))

        # Assemble labels that say "all real images"
        misleading_labels = tf.ones((batch_size, 1))

        # Train the generator (note that we should *not* update the weights of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(seed))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }


In [76]:
class SaveImageCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        seed = tf.random.normal(shape=(NUM_EXAMPLES_TO_GENERATE, LATENT_DIM))
        predictions = self.model.generator(seed, training=False)
        fig = plt.figure(figsize=(10, 10))
        for i in range(predictions.shape[0]):
            plt.subplot(4, 4, i + 1)
            plt.imshow(predictions[i] * 0.5 + 0.5)
            plt.axis("off")
        plt.savefig(f"{OUTPUT_DIR}/image_at_epoch_{epoch:04d}.png")
        plt.close(fig)

In [ ]:
# The optimizers for Generator and Discriminator
discriminator_opt = tf.keras.optimizers.legacy.Adam(learning_rate=5e-5, beta_1=0.5, clipnorm=1.0)
generator_opt = tf.keras.optimizers.legacy.Adam(learning_rate=1e-4, beta_1=0.5, clipnorm=1.0)
# To compute cross entropy loss
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)

# Defining GAN Model
model = GAN(discriminator=discriminator, generator=generator, latent_dim=LATENT_DIM)

# Compiling GAN Model
model.compile(d_optimizer=discriminator_opt, g_optimizer=generator_opt, loss_fn=loss_fn)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
# Fitting the GAN
history = model.fit(data, epochs=EPOCHS, callbacks=[SaveImageCallback(), tensorboard_callback])

model.save(f"{OUTPUT_DIR}/model_final.keras")
